In [1]:
%load_ext autoreload
%autoreload 2

### read data from text file and predict ner labels

In [2]:
with open('test_article2.txt') as file:
    lines = file.readlines()

In [3]:
data_tags = ['0','Metal', 'M-cont' , 'Element', 'E-cont', 'Acid', 'A-cont', 'Yield' , 'Y-cont', 'Separation Method' , 'S-cont', 'Resin', 'R-cont', 'Method of Analysis', 'T-cont', 'pH', 'P-cont', 'Chemical Compound', 'H-cont', 'Organic solvent', 'O-cont', 'Element Group', 'G-cont', 'Inorganic Solvent', 'I-cont', 'Flowrate', 'F-cont', 'Acid Concentration', 'C-cont', 'Reagent', 'X-cont']
tag_values = data_tags
print(tag_values)

tag_values.append("PAD")
print(tag_values)

tag2idx = {t: i for i, t in enumerate(tag_values)}
print(tag2idx)
    
idx2tag = {value: key for key, value in tag2idx.items()}

['0', 'Metal', 'M-cont', 'Element', 'E-cont', 'Acid', 'A-cont', 'Yield', 'Y-cont', 'Separation Method', 'S-cont', 'Resin', 'R-cont', 'Method of Analysis', 'T-cont', 'pH', 'P-cont', 'Chemical Compound', 'H-cont', 'Organic solvent', 'O-cont', 'Element Group', 'G-cont', 'Inorganic Solvent', 'I-cont', 'Flowrate', 'F-cont', 'Acid Concentration', 'C-cont', 'Reagent', 'X-cont']
['0', 'Metal', 'M-cont', 'Element', 'E-cont', 'Acid', 'A-cont', 'Yield', 'Y-cont', 'Separation Method', 'S-cont', 'Resin', 'R-cont', 'Method of Analysis', 'T-cont', 'pH', 'P-cont', 'Chemical Compound', 'H-cont', 'Organic solvent', 'O-cont', 'Element Group', 'G-cont', 'Inorganic Solvent', 'I-cont', 'Flowrate', 'F-cont', 'Acid Concentration', 'C-cont', 'Reagent', 'X-cont', 'PAD']
{'0': 0, 'Metal': 1, 'M-cont': 2, 'Element': 3, 'E-cont': 4, 'Acid': 5, 'A-cont': 6, 'Yield': 7, 'Y-cont': 8, 'Separation Method': 9, 'S-cont': 10, 'Resin': 11, 'R-cont': 12, 'Method of Analysis': 13, 'T-cont': 14, 'pH': 15, 'P-cont': 16, 'Chemi

In [4]:
def degreekify(char):
    #char will be a character
    greek = {'α': '[alpha]', 'β':'[beta]', 'γ': '[gamma]', 'δ':'[delta]', 'ε': '[epsilon]', 'ζ':'[zeta]', 'η': '[eta]',
            'θ':'[theta]', 'ι': '[iota]', 'κ':'[kappa]', 'λ':'[lambda]', 'μ': '[mu]', 'ν':'[nu]', 'ξ':'[xi]', 'ο':'[omicron]', 'π':'[pi]', 'ρ':'[rho]',
            'σ': '[sigma]', 'τ': '[tau]', 'υ':'[upsilon]', 'φ':'[phi]', 'χ':'[chi]', 'ψ':'[psi]', 'ω':'[omega]' }
    if char in greek:
        return greek[char]
    else:
        return char

In [5]:
def pre_process(text, annotations):
    #text will be the straight sentence, info[i]['text']
    #annotations will be the list of labels, must be info[i]['annotations']
    
    text_dict = []
    
    for i in range(len(text)):
        text_dict.append(degreekify(text[i]))
    
    
    
    ann_indices = []
    def fun(x):
        return x['start_offset']
    annotations.sort(key=fun)
    
    
    
    if len(annotations)==0:
        ann_indices.append([[0, len(text)],0])
    else:
        ann_indices.append([[0, annotations[0]['start_offset']], 0])
        for i in range(len(annotations)-1):
            ann_indices.append([[annotations[i]['start_offset'], annotations[i]['end_offset']], data_tags.index(annotations[i]['label'])])
            ann_indices.append([[annotations[i]['end_offset'], annotations[i+1]['start_offset']], 0])
            
        ann_indices.append([[annotations[-1]['start_offset'], annotations[-1]['end_offset']], data_tags.index(annotations[-1]['label'])])
        ann_indices.append([[annotations[-1]['end_offset'], len(text)], 0])
         
    
    labels = []
    sentences = []
    for a in ann_indices:
        
        if a[0][1]-a[0][0] !=0:
            together = ''
            for i in range(a[0][0], a[0][1]):
                together += text_dict[i]
                
            toke = together.split()
            sentences.extend(toke)
            t = len(toke)
            if t != 0:
                temp = [data_tags[a[1]+1]] * t
                if a[1] != 0:
                    temp[0] = data_tags[a[1]]
                labels.extend(temp)
 
    return labels, sentences

In [6]:
def reduce(sent, label, slist, llist):
    lens = len(sent)
    if lens < 256:
        slist.append(sent)
        llist.append(label)
    else:
        t = lens//2
        return reduce(sent[:t], label[:t], slist, llist), reduce(sent[t:], label[t:], slist, llist)

#### import tokenizer and model

In [7]:
from transformers import BertTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset
from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader
model = AutoModelForTokenClassification.from_pretrained(
    "ner_save_model_ssl",
    )
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_cased', do_lower_case=False)

2023-09-12 15:17:04.973871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
sentences, labels = [], []

for i in range(len(lines)):
    l, s = pre_process(lines[i], [])
    reduce(s,l,sentences, labels)

In [9]:
def tokenize_and_preserve_labels(sentence, text_labels): #### this function requires to add 
    # more work on how to add labels
    tokenized_inputs = tokenizer(sentence, truncation=True, is_split_into_words=True)
    #tokenized_sentence = []
    labels = [-100]
    word_idx_list = [0]
    
    widx = 1

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        
        n_subwords = len(tokenized_word)
        #print(word, tokenized_word, n_subwords)

        # Add the tokenized word to the final tokenized word list
        #tokenized_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_word]
        #tokenized_sentence.extend(tokenized_ids)

        # Add the same label to the new list of labels `n_subwords` times
        labels.append(tag2idx[label])
        word_idx_list.append(widx)
        if n_subwords>1:
            labels.extend([-100] * (n_subwords-1))
            word_idx_list.extend([widx]*(n_subwords-1))
        #labels.extend([tag2idx[label]] * n_subwords) #### TODO: this extend should change I to I-cont, but keep I-cont

        # tokenized_sentence2 = pad_sequences(tokenized_sentence,
        #                   maxlen=MAX_LEN, dtype="long", value=0.0,
        #                   truncating="post", padding="post")
        # labels2 = pad_sequences(labels,
        #              maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
        #              dtype="long", truncating="post")
        widx += 1
    
        
    labels.append(-100)
    word_idx_list.append(widx)
    return tokenized_inputs, labels, word_idx_list

def tokenize_text(example):
    sent = example['tokens']
    labs = example['tags']
    example, labels, wordidx = tokenize_and_preserve_labels(sent, labs)
    example['labels'] = labels
    example['wordidx'] = wordidx
    return example

In [10]:
def test_data_loaders_from_lines(lines):
    sentences, labels = [], []

    for i in range(len(lines)):
        l, s = pre_process(lines[i], [])
        reduce(s,l,sentences, labels)
        
    
    data_list = [{'tokens':x, 'tags':y} for x, y in zip(sentences, labels) ]
    text_data = Dataset.from_list(data_list)
    tokenized_data = text_data.map(tokenize_text)
    tokenized_data = tokenized_data.remove_columns(['tokens', 'tags'])
    


    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
    test_dataloader = DataLoader(tokenized_data, collate_fn=data_collator, shuffle=False)
    return test_dataloader

In [11]:
test_dataloader = test_data_loaders_from_lines(lines)

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
import torch
import numpy as np
device = torch.device("cuda")
token_label_pairs = []
for j, batch in enumerate(test_dataloader):

    model.to(device)
    batch.to(device)
    wordidx = batch.pop("wordidx")[0]
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs[1].detach().cpu().numpy()
    label_indices = np.argmax(logits, axis=2)
    tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'].to('cpu').numpy()[0])
    
    new_tokens, new_labels = [], []
    i = 0
    for token, label_idx in zip(tokens, label_indices[0]):
        if token.startswith("##"):
            new_tokens[-1] = new_tokens[-1] + token[2:]
        elif i>0 and wordidx[i] == wordidx[i-1]:
            new_tokens[-1] += token
        else:
            new_labels.append(tag_values[label_idx])
            new_tokens.append(token)
        i += 1
    new_tokens = new_tokens[1:-1]
    new_labels = new_labels[1:-1]
    temp_dict = dict()
    temp_dict['tokens'] = new_tokens
    temp_dict['labels'] = new_labels
    token_label_pairs.append(temp_dict)

In [41]:
import json 
with open('ner_result_no_E.json', "w") as json_file:
    json.dump(token_label_pairs, json_file, indent=4)

# generate html with ner results

In [13]:
import json
from ipymarkup import show_span_box_markup
with open('ner_result_no_E.json') as f:
    ner_dict = json.load(f)

In [43]:
idx2ner_labels = {'0': 0, 'Metal': 1, 'M-cont': 2, 'Element': 3, 'E-cont': 4, 'Acid': 5, 'A-cont': 6, 'Yield': 7, 'Y-cont': 8, 
            'Separation Method': 9, 'S-cont': 10, 'Resin': 11, 'R-cont': 12, 'Method of Analysis': 13, 'T-cont': 14, 'pH': 15, 
            'P-cont': 16, 'Chemical Compound': 17, 'H-cont': 18, 'Organic solvent': 19, 'O-cont': 20, 'Element Group': 21, 'G-cont': 22, 
            'Inorganic Solvent': 23, 'I-cont': 24, 'Flowrate': 25, 'F-cont': 26, 'Acid Concentration': 27, 'C-cont': 28, 'Reagent': 29, 'X-cont': 30, 'PAD': 31}

In [14]:
# ids = 4


# text = " ".join(tokens)
spans = []
start = 0
curr = 0
text = ''
for ids in range(len(ner_dict)):
    tokens = ner_dict[ids]['tokens']
    ner_labels = ner_dict[ids]['labels']
    text += " ".join(tokens)
    text +='\n'
    #curr += 1
    for i, token in enumerate(tokens):
        end = curr + len(tokens[i])
        if ner_labels[i] != '0':
            spans.append((curr, end, ner_labels[i]))
            # if i == 0:  # the first one must be the start label
            #     start = curr 
            # # spans.append((start, end, ner_labels[0]))
            # elif ner_labels[i] != ner_labels[i-1]:
            #     start = curr
            # if i+1== len(tokens) or ner_labels[i] != ner_labels[i+1]:
            #     spans.append((start, end, ner_labels[i]))
        curr = end+1
show_span_box_markup(text, spans)

In [25]:
from ipymarkup import format_span_box_markup
lines = format_span_box_markup(text, spans)
html = ''.join(lines)
with open('viz_no_E.html', 'w') as outf:
    _ = outf.write(html)